In [1]:
import keras
from keras import layers, backend
import pandas as pd
import numpy as np
from dfply import *
from plotnine import *
from sklearn.cluster import SpectralClustering
from numpy.random import seed
from tensorflow.random import set_seed as tf_set_seed

import bokeh.plotting as pl;
from bokeh.io import output_notebook;
import bokeh.models as bm;
import bokeh.layouts as bl;

output_notebook()

seed(1000);
tf_set_seed(1000);


from pandasql import sqldf
sql = lambda q: sqldf(q, globals())

2022-08-29 15:34:30.485665: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-29 15:34:30.485685: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Loading BokehJS ...

In [2]:
outcomes = pd.read_csv("derived_data/outcome_vectors.csv").fillna(-1)
columns_to_encode = ['-1','0','1','2','3','6','12'];
enc = keras.models.load_model("models/imputation-enc/")
vae = keras.models.load_model("models/imputation-vae/")

2022-08-29 15:34:36.087229: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-29 15:34:36.087247: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-29 15:34:36.087265: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (eda7cbb5e25c): /proc/driver/nvidia/version does not exist
2022-08-29 15:34:36.087722: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
encoded = pd.DataFrame(enc.predict(outcomes[columns_to_encode]), columns={"VAE1","VAE2"})
auto_encoded = pd.DataFrame(vae.predict(outcomes[columns_to_encode]), columns=columns_to_encode)
encoded['bpi_intensity'] = auto_encoded.apply(lambda x : [x[k] for k in columns_to_encode], axis=1)
encoded['time'] = auto_encoded.apply(lambda x : [int(k) for k in columns_to_encode], axis=1)
                                     
def fill_in_missing(raw_outcomes, auto_enc_outcomes):
    def f(x):
        y = auto_enc_outcomes.loc[x.name];
        for c in columns_to_encode:
            x[c] = x[c] if x[c] > 0 else y[c];
        return x;
    return raw_outcomes.apply(f, axis=1, result_type='broadcast');

filled_in = fill_in_missing(outcomes, auto_encoded);
encoded['bpi_intensity_imputed'] = filled_in.apply(lambda x : [x[k] for k in columns_to_encode], axis=1)
encoded['color'] = [ 'red' if group == 1 else 'green' if group == 2 else 'blue' for group in outcomes['group']]


5/5 [==============================] - 0s 2ms/step


In [4]:
tools = "box_select, lasso_select, help";
source = bm.ColumnDataSource(data=encoded);
p_embedding = pl.figure(width=400, height=400, tools=tools);
p_embedding.circle('VAE1','VAE2', size=3, color='color', alpha=0.5, source=source);

p_outcome = pl.figure(width=600, height=400, tools=tools);
p_outcome.multi_line('time', 'bpi_intensity_imputed', source=source, color='color');

p_outcome_enc = pl.figure(width=600, height=400, tools=tools);
p_outcome_enc.multi_line('time', 'bpi_intensity', source=source, color='color');


pl.show(bl.gridplot([[p_embedding, p_outcome, p_outcome_enc]]))

In [5]:
def fill_in_missing(raw_outcomes, auto_enc_outcomes):
    def f(x):
        y = auto_enc_outcomes.loc[x.name];
        for c in columns_to_encode:
            x[c] = x[c] if x[c] > 0 else y[c];
        return x;
    return raw_outcomes.apply(f, axis=1, result_type='broadcast');

filled_in = fill_in_missing(outcomes, auto_encoded);
encoded['bpi_intensity_imputed'] = filled_in.apply(lambda x : [x[k] for k in columns_to_encode], axis=1)


In [46]:
auto_encoded.loc[0]

-1    3.501688
0     2.499121
1     2.464190
2     2.441996
3     2.611718
6     2.502359
12    2.870914
Name: 0, dtype: float32

In [53]:
outcomes

,id,group,0,1,2,3,6,12,-1
0,12,3,2.75,-1.00,-1.00,-1.00,-1.00,-1.00,3.000
1,14,3,2.00,2.00,1.00,3.75,1.75,3.25,3.000
2,15,3,2.75,1.50,1.00,1.00,2.50,0.25,3.000
3,18,2,2.25,2.75,2.25,1.00,0.00,0.75,4.625
4,23,2,2.25,2.50,2.25,2.50,2.25,3.50,3.250
...,...,...,...,...,...,...,...,...,...
146,219,3,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,4.500
147,339,3,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,4.500
148,352,3,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,5.500
149,651,2,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,2.125


In [23]:
demo_to_outcome = keras.models.load_model("models/demo_embedding_to_outcome/")
demo_embedding = pd.read_csv("derived_data/demographic_ae.csv")
demo_embedding = sql('''select o.id, o.`group`, d.AE1, d.AE2 from outcomes as o
join demo_embedding as d on d.id = o.id''')
model_input = sql('''
select case when `group` == 1 then 1 else 0 end as g1,
 case when `group` == 2 then 1 else 0 end as g2,
 case when `group` == 3 then 1 else 0 end as g3,
 AE1,
 AE2 from demo_embedding''');
predicted = pd.DataFrame(demo_to_outcome.predict(model_input[['g1','g2','g3','AE1','AE2']]),columns=columns_to_encode)
demo_embedding['predicted_bpi'] = predicted.apply(lambda x: [x[k] for k in columns_to_encode], axis=1);
demo_embedding['actual_bpi'] = encoded['bpi_intensity_imputed'];
demo_embedding['color'] = [ 'red' if group == 1 else 'green' if group == 2 else 'blue' for group in demo_embedding['group']]
demo_embedding['time'] = predicted.apply(lambda x: [int(k) for k in columns_to_encode], axis=1);


5/5 [==============================] - 0s 2ms/step


In [28]:
tools = "box_select, lasso_select, help, box_zoom";
source = bm.ColumnDataSource(data=demo_embedding);
p_embedding = pl.figure(width=400, height=400, tools=tools);
p_embedding.circle('AE1','AE2', size=3, color='color', alpha=0.5, source=source);

p_outcome = pl.figure(width=600, height=400, tools=tools);
p_outcome.multi_line('time', 'predicted_bpi', source=source, color='color');

p_real = pl.figure(width=600, height=400, tools=tools);
p_real.multi_line('time', 'actual_bpi', source=source, color='color');

pl.show(bl.gridplot([[p_embedding, p_outcome, p_real]]))

In [8]:
demo_to_outcome = keras.models.load_model("models/demo_embedding_to_outcome/")
